***Welcome to Medical image processing in Python***<br/>

Presented by: Reza Saadatyar (2024-2025) <br/>
E-mail: Reza.Saadatyar@outlook.com 

**Import the require library**

In [1]:
import sys
import numpy as np
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from colorama import Back, Fore

# from skimage.viewer import ImageViewer
# from OOP import Pre_Processing_R2g
# from Functions import image_resizer

**1. Set Image Path & Load data**

In [7]:
import os
from itertools import chain

class DirectoryReader:
    """
    A class for reading and processing files in a directory with a specific format.
    """
    def __init__(self, directory_path: str, format_type: str) -> None:
        """
        Initialize the DirectoryReader with a directory path and file format.

        :param directory_path: Path to the directory to scan for files.
        :param format_type: File format (extension) to filter files, e.g., ".tif".
        """        
        self.files: list[str] = [] # Stores the names of all files matching the specified format.
        self.full_path: list[str] = [] # Stores the full paths of all matching files.
        self.folder_path: list[str] = [] # Stores the unique folder paths containing the files.
        self.subfolder: list[list[str]] = [] # Stores the names of subfolders for each directory.
        self.format_type: str = format_type # Stores the file format to filter (e.g., ".tif").
        self.directory_path: str = directory_path # Stores the root directory path to scan.
        self._scan_directory()  # Perform the directory scanning process.

    def _scan_directory(self) -> None:
        for root, subfolder_name, files_name in os.walk(self.directory_path):  # Traverse the directory tree
            root = root.replace("\\", "/")  # Replace backslashes with forward slashes for cross-platform compatibility

            for file in files_name:
                if file.endswith(self.format_type):  # Check if the file ends with the specified format
                    self.files.append(file)  # Append the file name to the files list

                    if root not in self.folder_path:  # Check if the root folder is not already in the folder_paths list
                        self.folder_path.append(root)  # If not, append the root folder to the folder_paths list

                    self.full_path.append(os.path.join(root, file).replace("\\", "/"))  # Append the full file path

                    # Ensure subfolder names are unique and non-empty
                    if subfolder_name not in self.subfolder and subfolder_name != []:
                        self.subfolder.append(subfolder_name)  # Append subfolder names to subfolders list

    @property
    def all_file_paths(self) -> list[str]:
        """
        Retrieve all full file paths for files with the specified format.

        :return: List of full file paths.
        """
        return self.full_path

    @property
    def filenames(self) -> list[str]:
        """
        Retrieve the list of filenames.

        :return: List of filenames.
        """
        return self.files

    @property
    def folder_paths(self) -> list[str]:
        """
        Retrieve the list of folder paths containing the files.

        :return: List of folder paths.
        """
        return self.folder_path

    @property
    def subfoldernames(self) -> list[str]:
        """
        Retrieve a flattened list of subfolder names.

        :return: Flattened list of subfolder names.
        """
        return list(chain.from_iterable(self.subfolder))

In [8]:
directory_path = "D:/Medical-Image-Processing/Data/Inputs"

# Create an instance of DirectoryReader with the directory path and file format
file_reader = DirectoryReader(directory_path, format_type="tif")

path__with_file = file_reader.all_file_paths  # Get the list of all file paths
folder_paths = file_reader.folder_paths  # Get the list of folder paths containing the files
files = file_reader.filenames  # Get the list of filenames
subfolders = file_reader.subfoldernames  # Get the flattened list of subfolder names

print(Fore.GREEN + f"{path__with_file = }""\n" + Fore.BLUE + f"{folder_paths = }""\n" + Fore.MAGENTA + f"{files = }"+
      "\n" + Fore.CYAN + f"{subfolders = }")

path__with_file = ['D:/Medical-Image-Processing/Data/Inputs/ytma12_010804_benign2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma12_010804_benign3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma12_010804_malignant1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma12_010804_malignant2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma12_010804_malignant3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma23_022103_benign1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma23_022103_benign2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma23_022103_benign3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma23_022103_malignant1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma23_022103_malignant2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma23_022103_malignant3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma49_042003_benign1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs/ytma49_042003_benign2_ccd.tif', 'D:/Medical-Im

*Image-Width: n*<br/>
*Image-Height: m*<br/>
*Channels: c*<br/>
*Planes: p*<br/>
*Grayscale: (p, m, n)*<br/>
*RGB: (p, m, n, c)*<br/>

**2. Convert the image into an array**

In [15]:
import numpy as np  # Import numpy for array manipulation
from skimage import io  # Import scikit-image library for image I/O operations
from Functions import directory_reader

class ImageLoader:
    """
    A class to load and process images from a directory into a NumPy array.
    """

    def __init__(self, directory_path: str, format_type: str) -> None:
        """
        Initialize the ImageLoader class.

        :param directory_path: Root directory containing the images.
        :param format_type: File format (extension) of the images (e.g., ".tif").
        """
        self.images = None  # Placeholder for loaded image array
        self.img_width = None  # Width of the images
        self.img_height = None  # Height of the images
        self.img_channels = None  # Number of color channels in the images
        self.format_type: str = format_type  # File format to filter (e.g., ".tif")
        self.directory_path: str = directory_path  # Root directory path to scan

    @property
    def load_dataset(self) -> np.ndarray:
        """
        Load images from the specified directory into a NumPy array.

        :return: NumPy array containing the loaded images.
        """
        # Initialize the DirectoryReader to get file paths
        dir_reader = directory_reader.DirectoryReader(self.directory_path, self.format_type)
        all_path = dir_reader.all_file_paths

        if not all_path:
            raise ValueError("No files found in the specified directory.")

        num_files = len(all_path)  # Total number of image files
        first_image = io.imread(all_path[0])  # Load the first image to determine its dimensions

        # Check if the image is grayscale or colored
        if first_image.ndim == 2:  # Grayscale image
            self.img_height, self.img_width = first_image.shape  # Get image dimensions
            self.images = np.zeros(  # Initialize NumPy array for grayscale images
                (num_files, self.img_height, self.img_width),
                dtype=np.uint8  # Pixel values are stored as unsigned 8-bit integers
            )
        else:  # Colored image (e.g., RGB)
            self.img_height, self.img_width, self.img_channels = first_image.shape  # Get image dimensions
            self.images = np.zeros(  # Initialize NumPy array for colored images
                (num_files, self.img_height, self.img_width, self.img_channels),
                dtype=np.uint8
            )

        # Load all images into the NumPy array
        for idx, file_path in enumerate(all_path):
            self.images[idx] = io.imread(file_path)  # Read and store each image

        return self.images  # Return the loaded images as a NumPy array

**2.1. Convert the images in the "Inputs" folder to a NumPy array**

In [16]:
file_path = "D:/Medical-Image-Processing/Data/Inputs/"
inputs = ImageLoader(file_path, format_type="tif") # A class to load and process images from file paths into a NumPy array.
inputs_array = inputs.load_dataset # Load images from the provided file paths into a NumPy array.
print(Fore.GREEN + f"{inputs_array.shape = }") 

inputs_array.shape = (58, 768, 896, 3)


In [17]:
file_path = "D:/Medical-Image-Processing/Data/Masks/"
masks = ImageLoader(file_path, format_type="TIF") # A class to load and process images from file paths into a NumPy array.
masks_array = masks.load_dataset # Load images from the provided file paths into a NumPy array.
print(Fore.GREEN + f"{masks_array.shape = }") 

masks_array.shape = (58, 768, 896)


**2.2. Convert the images in the "Masks" folder to boolean**

In [23]:
file_path = "D:/Medical-Image-Processing/Data/Masks/"
dir_reader = directory_reader.DirectoryReader(file_path, format_type="TIF")
all_path = dir_reader.all_file_paths

num_files = len(all_path)  # Total number of image files
first_image = io.imread(all_path[0])  # Load the first image to determine its dimensions

img_height = first_image.shape[0]         # Get the height of the image
img_width = first_image.shape[1]          # Get the width of the image
labels = np.zeros((len(all_path), img_height, img_width, 2), dtype = bool)  # Shape: [num_files, H, W, 2]

sys.stdout.flush()
for ind, val in tqdm(enumerate(all_path)):  # Progressively iterate through all the input files
    mask = np.squeeze(io.imread(val)).astype(bool)  # Load and convert each mask to boolean
    labels[ind, :, :, 0] = ~mask  # Background (inverse of mask)
    labels[ind, :, :, 1] = mask   # Foreground (actual mask)

58it [00:00, 73.88it/s]


**3. Image Processor**<br/>
***3.1. [Image Resizing](https://scikit-image.org/docs/stable/api/skimage.transform.html#skimage.transform.resize)***<br/>
- `Standardizing Image Dimensions:` Machine learning models, like CNNs, need input data with fixed dimensions. For instance, if a model requires images of size 224x224x3, all input images must be resized to that shape.<br/>
- `Reducing Computational Load:`Resizing images to smaller dimensions lowers computational costs, particularly with large datasets, and aids in faster training or inference for deep learning models.

**Augmentation, Re_Color, & Im_Saving**<br/>

In [115]:
import os
import glob
import shutil
import numpy as np
from tensorflow import keras
from Functions import directory_reader
from skimage import io, color, transform  # Import transform module for image resizing

# ================================= Class for resizing & converting images ====================================
class ImageProcessor:
    """
    A class for processing image data, including resizing.
    """
    def __init__(self):
        """
        Initialize the ImageProcessor class.

        :param img_data: A numpy array of shape [num_images, height, width, channels].
        """
        self.img_gray = None  # Placeholder for grayscale images after processing
        self.file_names = None  # Placeholder for image file names
        self.file_path = None  # Placeholder for the path to the image files
        self.augmente_path = None  # Placeholder for the path to store augmented images
        
    # ---------------------------------------------- Resizes ---------------------------------------------------
    def resize_images(self, data: np.ndarray, img_height_resized: int, img_width_resized: int) -> np.ndarray:
        """
        Resizes a batch of images to the specified height, width, and channels.

        :param img_height_resized: Target height of the images after resizing.
        :param img_width_resized: Target width of the images after resizing.
        :return: A numpy array of resized images.
        """

        # Check if the input is grayscale (3D) or colored (4D)
        if data.ndim == 3:  # Grayscale images (no color channels)
            img_channels = 1  # Grayscale implies 1 channel
            # Initialize an array to store resized grayscale images
            resized_imgs = np.zeros(
                (data.shape[0], img_height_resized, img_width_resized),
                dtype=np.uint8
            )

            # Loop through each grayscale image in the batch
            for i in range(data.shape[0]):
                # Resize the image to the target dimensions and store it
                resized_imgs[i] = transform.resize(
                    self.img_data[i], (img_height_resized, img_width_resized),
                    preserve_range=True  # Preserve the range of pixel values
                )

        else:  # Colored images (4D array with channels)
            img_channels = data.shape[-1]  # Get the number of color channels
            # Initialize an array to store resized colored images
            resized_imgs = np.zeros(
                (data.shape[0], img_height_resized, img_width_resized, img_channels),
                dtype=np.uint8
            )

            # Loop through each colored image in the batch
            for i in range(data.shape[0]):
                # Resize the image to the target dimensions and store it
                resized_imgs[i] = transform.resize(
                    data[i], (img_height_resized, img_width_resized, img_channels),
                    preserve_range=True  # Preserve the range of pixel values
                )

        return resized_imgs  # Return the resized images
    
    # ---------------------------------------- RGB images to grayscale -----------------------------------------
    def RGB2Gray(self, file_path: str, format_type: str) -> np.ndarray:
        """
        Convert RGB images in the specified directory to grayscale.

        :param file_path: Path to the directory containing image files.
        :param format_type: File format (e.g., ".jpg", ".png") to filter images.
        :return: A NumPy array containing grayscale images.
        """

        # Initialize the DirectoryReader to get file paths
        dir_reader = directory_reader.DirectoryReader(file_path, format_type)
        all_path = dir_reader.all_file_paths # Retrieve all file paths in the specified format
        self.file_names = dir_reader.filenames # Store filenames in the class attribute for further reference

        if not all_path: # Raise an error if no files are found
            raise ValueError("No files found in the specified directory.")

        img_num = len(all_path) # Get the total number of images

        # Retrieve the dimensions of the first image to initialize the grayscale array
        img_height, img_width, _ = io.imread(all_path[0]).shape

        # Initialize a NumPy array to store grayscale images
        self.img_gray = np.zeros((img_num, img_height, img_width), dtype=np.uint8)

        for ind, val in enumerate(all_path): # Convert each image to grayscale
            # Read the image, convert it to grayscale, scale back to [0, 255], and store it in the array
            self.img_gray[ind] = (color.rgb2gray(io.imread(val)) * 255).astype(np.uint8)

        return self.img_gray # Return the array of grayscale images
    
    # ------------------------------------------------- Save image ---------------------------------------------
    def save_img_gray(self, path_save):
        """
        Save grayscale images to a specified directory.

        :param path_save: Path to the directory where the grayscale images will be saved.
        """

        # Create a folder named 'Gray image/' inside the specified save path, if it doesn't already exist
        os.makedirs(os.path.join(path_save, 'Gray image/'), exist_ok=True)

        # Loop through each image and its corresponding filename
        for ind, filename in enumerate(self.file_names):
            # Save each grayscale image to the 'Gray image/' folder using its original filename
            io.imsave(fname='{}{}'.format(path_save + 'Gray image/', filename), arr=self.img_gray[ind])

        print(Fore.GREEN + "The images have been saved successfully.") # Print a success message to the console

    #  ----------------------------------------------- Augmentation --------------------------------------------
    def augmentation(self, file_path: str, augmente_path: str, num_augmented_imag: int, rotation_range: int,
                     format_type: str) -> None:
        """
        Applies image augmentation (rotation) to images in the specified directory and saves them.

        :param file_path: Path to the directory containing the images.
        :param augmente_path: Path to the directory to save augmented images.
        :param num_augmented_imag: Number of augmented images to generate.
        :param rotation_range: Degree range for random image rotation.
        :param format_type: File format to filter images (e.g., ".jpg", ".png").
        """
        self.file_path = file_path  # Store the input file path
        self.augmente_path = augmente_path  # Store the augmented images save path
        self.augmente_path = os.path.join(self.augmente_path, 'Rotated/')  # Create a subfolder for rotated images

         # Check if the augmented images folder exists, delete it if so
        if os.path.exists(self.augmente_path): shutil.rmtree(self.augmente_path)# Delete the folder and its contents
        os.makedirs(self.augmente_path, exist_ok=True)  # Recreate the folder

        # Create a temporary folder inside the input directory for processing
        TEMP_DIR = os.path.join(self.file_path, 'Temp/')
        if os.path.exists(TEMP_DIR): shutil.rmtree(TEMP_DIR)  # Delete the temporary folder if it exists
        os.makedirs(TEMP_DIR, exist_ok=True)  # Recreate the temporary folder
        
        # Initialize the DirectoryReader to get file paths
        dir_reader = directory_reader.DirectoryReader(self.file_path, format_type)
        all_path = dir_reader.all_file_paths  # Get all file paths matching the format
        file_names = dir_reader.filenames  # Get corresponding filenames
        
         # Raise an error if no files are found in the directory
        if not all_path: raise ValueError("No files found in the specified directory.")
        
        dat = io.imread(all_path[0])  # Read the first image to determine its dimensions

        # Copy all files from the main folder to the temporary folder
        for ind, val in enumerate(all_path):
            shutil.copy(val, os.path.join(TEMP_DIR, file_names[ind]))

        # Set up the ImageDataGenerator for image augmentation
        Data_Gen = keras.preprocessing.image.ImageDataGenerator(rotation_range=rotation_range)
        # Use flow_from_directory to process the images in the Temp folder
        img_aug = Data_Gen.flow_from_directory(
            self.file_path,     # Parent directory of Temp
            classes=['Temp'],         # Specify the subfolder 'Temp' as the target
            batch_size=1,             # Process one image at a time
            save_to_dir=self.augmente_path,  # Save augmented images to the Rotated folder
            save_prefix='Aug',        # Prefix for augmented images
            target_size=(dat.shape[0], dat.shape[1]),  # Resize images to the specified dimensions
            class_mode=None           # No labels, as we're working with unclassified images
        )
        
        for _ in range(num_augmented_imag):  # Generate augmented images and save them
            next(img_aug)  # Process the next image and save it

        shutil.rmtree(TEMP_DIR)  # Delete the temporary folder and its contents after processing



**3.1. Image Resizing**

In [116]:
img = ImageProcessor()   # Create an instance of the ImageProcessor class

# Call the `resize_images` method to resize the images to the target dimensions (255x255)
resized_images = img.resize_images(inputs_array, img_height_resized=255, img_width_resized=255)  # Resize all images to 255x255
print(Fore.GREEN + f"Resizing images from {inputs_array.shape} to {resized_images.shape}")

Resizing images from (58, 768, 896, 3) to (58, 255, 255, 3)


**3.2. Convert RGB into Gray**

In [117]:
file_path = "D:/Medical-Image-Processing/Data/Inputs/"
img = ImageProcessor()
output_gray = img.RGB2Gray(file_path, format_type="tif")
print(Fore.GREEN + f"{output_gray.shape = }")

output_gray.shape = (58, 768, 896)


**3.3. Save images gray**

In [118]:
path_save = 'D:/Medical-Image-Processing/Data/'
img.save_img_gray(path_save)

The images have been saved successfully.


**3.4. Augmentation**

In [119]:
rotation_range = 30
num_augmented_imag = 3
# Path to the folder containing the original images
file_path = 'D:/Medical-Image-Processing/Data/Inputs/'

# Path where augmented images will be saved
augmente_path = 'D:/Medical-Image-Processing/Data/'
img = ImageProcessor()
img.augmentation(file_path, augmente_path, num_augmented_imag, rotation_range, format_type="tif")

Found 58 images belonging to 1 classes.


In [ ]:
# def plot_img_original_augment(self, num_img: int) -> None:
        
#         _, axs = plt.subplots(nrows=2, ncols=num_img)
#          # Check if num_img is 1 (special case for 1 image)
#         if num_img == 1:
#             # Display images on the first row
#             # io.imread(files_inputs[0])[:, :, :3].shape
#             axs[0].imshow(io.imread(glob.glob(self.file_path + '/*')[0]), cmap='gray')
#             axs[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Hide ticks
#             [spine.set_visible(False) for spine in axs[0].spines.values()]  # Hide all spines
#             axs[0].set_ylabel("Original Images", fontsize=12, labelpad=10)  # Y-axis label for the first row
            
#             # Display images on the second row
#             axs[1].imshow(io.imread(glob.glob(self.ugmente_path + '/*')[0]), cmap='gray')
#             axs[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Hide ticks
#             [spine.set_visible(False) for spine in axs[1].spines.values()]  # Hide all spines
            # axs[1].set_ylabel("Augmented Images", fontsize=12, labelpad=10)  # Y-axis label for the second row

    #     else:
    #         for i in range(num_img):
    #             # Display images on the first row
    #             axs[0, i].imshow(io.imread(glob.glob(self.imag_files_path + '/*')[i]), cmap='gray')
    #             axs[0, i].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Hide ticks
    #             [spine.set_visible(False) for spine in axs[0, i].spines.values()]  # Hide all spines

    #             # Display images on the second row
    #             axs[1, i].imshow(io.imread(glob.glob(self.imag_augmented_path + '/*')[i]), cmap='gray')
    #             axs[1, i].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Hide ticks
    #             [spine.set_visible(False) for spine in axs[1, i].spines.values()]  # Hide all spines

    #         # Add ylabel for each row (only set ylabel for the first column of each row)
    #         axs[0, 0].set_ylabel("Original Images", fontsize=12, labelpad=10)  # Y-axis label for the first row
    #         axs[1, 0].set_ylabel("Augmented Images", fontsize=12, labelpad=10)  # Y-axis label for the second row

    #     # Adjust layout to make sure images and titles don't overlap
    #     plt.tight_layout()

    #     # Auto-scale to fit the images in the figure area
    #     plt.autoscale(enable=True, axis='both', tight=True)
    #     plt.show()
        
# from skimage import transform, color  
